In [1]:
import re
import os
from glob import glob

import math
import numpy as np
import datetime
import pandas as pd

import matplotlib.pyplot as plt

import MDAnalysis as mda 

from MDAnalysis.coordinates.LAMMPS import DumpReader
from MDAnalysis import transformations as mdatransform
from MDAnalysis.analysis import diffusionmap, align, rms

from PipeLine import *

In [ ]:
mda.__version__

In [3]:
path='/Users/amirhsi_mini/N2000epsilon5.0r15.5lz379.5sig6.0nc1068dt0.005bdump1000adump5000ens*'
#path='/Users/amirhsi_mini/N1000epsilon5.0r8.0lz308.5sig2.0nc10412dt0.005bdump1000adump5000ens*'
geom = 'cylindrical'
trj_files = glob(path+"/N*all*")
all_pairs = PipeLine.file_reader(trj_files)


Total number of files is  8
Path to the first tuple of the sorted file:  ('/Users/amirhsi_mini/N2000epsilon5.0r15.5lz379.5sig6.0nc1068dt0.005bdump1000adump5000ens1/N2000epsilon5.0r15.5lz379.5sig6.0nc1068dt0.005bdump1000adump5000ens1.all.lammpstrj', '/Users/amirhsi_mini/N2000epsilon5.0r15.5lz379.5sig6.0nc1068dt0.005bdump1000adump5000ens1/N2000epsilon5.0r15.5lz379.5sig6.0nc1068dt0.005bdump1000adump5000ens1.all.data')


## distributions around center of simulation box with a user-defined cut_off

In [ ]:
all_data = all_pairs[0][1]
all_trj = all_pairs[0][0]
geometry = 'cylindrical'
rbin_factor = 0.1
zbin_factor = 0.5


print("Setting the name of analyze file...\n")
today = datetime.date.today().strftime('%Y%m%d')
cellAttrs = PipeLine.cellAttributes(all_trj,geometry, splitter='.lammpstrj')
time_unit = 1.0 # time unit = dmon * sqrt(mmon * epmon)
lj_nstep = cellAttrs.bdump # Sampling steps via dump command in Lammps
lj_dt = cellAttrs.dt # Timestep during the sampling process in Lammps
simulation_dt = lj_nstep * lj_dt * time_unit

cell = mda.Universe(all_data, all_trj, format = 'AMIRLAMMPSDUMP', atom_style = "id resid type x y z", dt = simulation_dt)

length_step = 10 * max(cellAttrs.dmon, cellAttrs.dcrowd)

#crds = cell.select_atoms('resid 0') # resid 1 is the atoms creating the chain, resid 0 is crowders

near_center = cell.select_atoms(f'prop abs z <= {length_step}', updating=True)

#crds = cell.select_atoms('prop abs z <= 50 resid 1', updating=True) # resid 1 is the atoms creating the chain, resid 0 is crowders
crds = near_center.select_atoms('resid 0', updating=True)

#chrm = cell.select_atoms('resid 1') # resid 1 is the atoms creating the chain, resid 0 is crowders
#chrm = cell.select_atoms('prop abs z <= 50 resid 1', updating=True) # resid 1 is the atoms creating the chain, resid 0 is crowders
chrm = near_center.select_atoms('resid 1', updating=True)
print ("Caution:The histograms are for the cylindrical coordinate system in the geometry=cylindrical. For other goemteries you need to redfined them.")
# Monomer local (number) density: 
# radial direction of the cylindrical coordinate system
#bin_size = cellAttrs.dcrowd / 2.
bin_size = rbin_factor * min(cellAttrs.dmon, cellAttrs.dcrowd)
r_sim_name = cellAttrs.filename.split("ens")[0]+'binFactor'+str(rbin_factor)+'ens'+cellAttrs.filename.split("ens")[1]
print(f"Bin size for r in the cylindrical geometry is set to {rbin_factor}*min(cellAttrs.dmon,cellAttrs.dcrowd)={bin_size} in a_m=1 units.")
lmax = 0.5 * cellAttrs.dcyl
#lmax = 0.5 * (cellAttrs.dcyl - max(cellAttrs.dmon, cellAttrs.dcrowd))
edge_name = 'rEdgesCrd'
redges, rhists_crd = PipeLine.bin_ceate(r_sim_name, bin_size, 0.0, lmax,edge_name)
edge_name = 'rEdgesMon'
_ , rhists_mon = PipeLine.bin_ceate(r_sim_name, bin_size, 0.0, lmax, edge_name)

# z direction of the cylindrical coordinate system
#bin_size = cellAttrs.dcrowd / 2.
bin_size = zbin_factor * min(cellAttrs.dmon, cellAttrs.dcrowd)
z_sim_name = cellAttrs.filename.split("ens")[0]+'binFactor'+str(zbin_factor)+'ens'+cellAttrs.filename.split("ens")[1]
print(f"Bin size for z in the cylindrical geometry is set to {zbin_factor}*min(cellAttrs.dmon,cellAttrs.dcrowd)={bin_size} in a_m=1 units.")
lmax = length_step
#lmax = 0.5 * (cellAttrs.lcyl - max(cellAttrs.dmon, cellAttrs.dcrowd))
edge_name = 'zEdgesCrd'
zedges, zhists_crd = PipeLine.bin_ceate(z_sim_name, bin_size, -1.0 * lmax, lmax, edge_name)
#zedges, zhists_crd = bin_ceate(z_sim_name, bin_size, 0, lmax, edge_name)
edge_name = 'zEdgesMon'
_ , zhists_mon = PipeLine.bin_ceate(z_sim_name, bin_size, -1.0 * lmax, lmax,edge_name)
#_ , zhists_mon = bin_ceate(z_sim_name, bin_size, 0, lmax,edge_name)

# theta of the cylindrical coordinate system
bin_size = np.degrees(np.pi/36) # is 6 degrees
theta_sim_name = cellAttrs.filename.split("ens")[0]+'binFactor1.0ens'+cellAttrs.filename.split("ens")[1]
print(f"Bin size for theta in the cylindrical geometry is set to 1.0*min(cellAttrs.dmon,cellAttrs.dcrowd)={bin_size} in a_m=1 units.")
edge_name = 'thetaEdgesCrd'
thetaedges, thetahists_crd = PipeLine.bin_ceate(theta_sim_name, bin_size, -1*np.degrees(np.pi), np.degrees(np.pi)-bin_size, edge_name)
edge_name = 'thetaEdgesMon'
_ , thetahists_mon = PipeLine.bin_ceate(theta_sim_name, bin_size, -1*np.degrees(np.pi), np.degrees(np.pi)-bin_size, edge_name)

# check if any of the histograms are empty or not.
if any([rhists_crd.any() != 0, rhists_mon.any() != 0, zhists_crd.any() != 0, zhists_mon.any() != 0, thetahists_crd.any() != 0, thetahists_mon.any() != 0]):
    raise ValueError("One of the histogram collectors are not empty!")

for ts in cell.trajectory: # the length of the for loop is equal to number of snapshots (configurations or frames)
    #number density in the cell's frame of reference
    # histogram in r direction
    rpos = np.linalg.norm(crds.positions[:,:2], axis = 1) # r component of position of each crowder
    dummy_hist, _ = np.histogram(rpos, redges)
    rhists_crd = np.add(rhists_crd, dummy_hist)

    rpos = np.linalg.norm(chrm.positions[:,:2], axis = 1) # r component of position of each monomer
    dummy_hist, _ = np.histogram(rpos, redges)
    rhists_mon = np.add(rhists_mon, dummy_hist)

    # histogram in z direction
    zpos = crds.positions[:,2] # z component of position of each crowder
    #zpos = np.abs(crds.positions[:,2]) # absolute value of the z component of position of each crowder - why absolute values? Bacause of PBC along z.
    dummy_hist, _ = np.histogram(zpos, zedges)
    zhists_crd = np.add(zhists_crd, dummy_hist)

    zpos = chrm.positions[:,2] # z component of position of each monomer
    #zpos = np.abs(chrm.positions[:,2]) # absolute value of the z component of position of each monomer - why absolute values? Bacause of PBC along z.
    dummy_hist, _ = np.histogram(zpos, zedges)
    zhists_mon = np.add(zhists_mon, dummy_hist)

    # histogram in theta 
    theta = np.degrees(np.arctan2(crds.positions[:,1], crds.positions[:,0])) # in degrees
    dummy_hist, _ = np.histogram(theta, thetaedges)
    thetahists_crd = np.add(thetahists_crd, dummy_hist)

    theta = np.degrees(np.arctan2(chrm.positions[:,1], chrm.positions[:,0])) # in degrees
    dummy_hist, _ = np.histogram(theta, thetaedges)
    thetahists_mon = np.add(thetahists_mon, dummy_hist)

lastname = 'Crd'
np.savetxt(r_sim_name+'_rHists'+lastname+'.csv', rhists_crd, delimiter = ',')
np.savetxt(z_sim_name+'_zHists'+lastname+'.csv', zhists_crd, delimiter = ',')
np.savetxt(theta_sim_name+'_thetaHists'+lastname+'.csv', thetahists_crd, delimiter = ',')

lastname = 'Mon'
np.savetxt(r_sim_name+'_rHists'+lastname+'.csv', rhists_mon, delimiter = ',')
np.savetxt(z_sim_name+'_zHists'+lastname+'.csv', zhists_mon, delimiter = ',')
np.savetxt(theta_sim_name+'_thetaHists'+lastname+'.csv', thetahists_mon, delimiter = ',')
print('done.')

## distributions around center of geometry of polymer with a user defined cut_off

In [ ]:
all_data = all_pairs[0][1]
all_trj = all_pairs[0][0]
geometry = 'cylindrical'
rbin_factor = 0.1
zbin_factor = 0.5


print("Setting the name of analyze file...\n")
today = datetime.date.today().strftime('%Y%m%d')
cellAttrs = PipeLine.cellAttributes(all_trj,geometry, splitter='.lammpstrj')
time_unit = 1.0 # time unit = dmon * sqrt(mmon * epmon)
lj_nstep = cellAttrs.bdump # Sampling steps via dump command in Lammps
lj_dt = cellAttrs.dt # Timestep during the sampling process in Lammps
simulation_dt = lj_nstep * lj_dt * time_unit

cell = mda.Universe(all_data, all_trj, format = 'AMIRLAMMPSDUMP', atom_style = "id resid type x y z", dt = simulation_dt)

length_step = 10 * max(cellAttrs.dmon, cellAttrs.dcrowd)

#externalRadius = 0.5 * cellAttrs.dcyl
zMax = length_step # the radii of the three types of particle.
#zMin  = -1 * length_step # the radii of the three types of particle.
#near_chrm_cog = cell.select_atoms(f'cyzone {externalRadius} {zMax} {zMin} resid 1', updating=True)
near_chrm_cog = cell.select_atoms(f'around {zMax} resid 1', updating=True)
#near_center = cell.select_atoms(f'prop abs z <= {length_step}', updating=True)

#crds = cell.select_atoms('resid 0') # resid 1 is the atoms creating the chain, resid 0 is crowders
#crds = cell.select_atoms('prop abs z <= 50 resid 1', updating=True) # resid 1 is the atoms creating the chain, resid 0 is crowders
crds = near_chrm_cog.select_atoms('resid 0', updating=True)
#chrm = cell.select_atoms('resid 1') # resid 1 is the atoms creating the chain, resid 0 is crowders
#chrm = cell.select_atoms('prop abs z <= 50 resid 1', updating=True) # resid 1 is the atoms creating the chain, resid 0 is crowders
chrm = near_chrm_cog.select_atoms('resid 1', updating=True)
print ("Caution:The histograms are for the cylindrical coordinate system in the geometry=cylindrical. For other goemteries you need to redfined them.")
# Monomer local (number) density: 
# radial direction of the cylindrical coordinate system
#bin_size = cellAttrs.dcrowd / 2.
bin_size = rbin_factor * min(cellAttrs.dmon, cellAttrs.dcrowd)
r_sim_name = cellAttrs.filename.split("ens")[0]+'binFactor'+str(rbin_factor)+'ens'+cellAttrs.filename.split("ens")[1]
print(f"Bin size for r in the cylindrical geometry is set to {rbin_factor}*min(cellAttrs.dmon,cellAttrs.dcrowd)={bin_size} in a_m=1 units.")
lmax = 0.5 * cellAttrs.dcyl
#lmax = 0.5 * (cellAttrs.dcyl - max(cellAttrs.dmon, cellAttrs.dcrowd))
edge_name = 'rEdgesCrd'
redges, rhists_crd = PipeLine.bin_ceate(r_sim_name, bin_size, 0.0, lmax,edge_name)
edge_name = 'rEdgesMon'
_ , rhists_mon = PipeLine.bin_ceate(r_sim_name, bin_size, 0.0, lmax, edge_name)

# z direction of the cylindrical coordinate system
#bin_size = cellAttrs.dcrowd / 2.
bin_size = zbin_factor * min(cellAttrs.dmon, cellAttrs.dcrowd)
z_sim_name = cellAttrs.filename.split("ens")[0]+'binFactor'+str(zbin_factor)+'ens'+cellAttrs.filename.split("ens")[1]
print(f"Bin size for z in the cylindrical geometry is set to {zbin_factor}*min(cellAttrs.dmon,cellAttrs.dcrowd)={bin_size} in a_m=1 units.")
lmax = length_step
#lmax = 0.5 * (cellAttrs.lcyl - max(cellAttrs.dmon, cellAttrs.dcrowd))
edge_name = 'zEdgesCrd'
zedges, zhists_crd = PipeLine.bin_ceate(z_sim_name, bin_size, -1.0 * lmax, lmax, edge_name)
#zedges, zhists_crd = bin_ceate(z_sim_name, bin_size, 0, lmax, edge_name)
edge_name = 'zEdgesMon'
_ , zhists_mon = PipeLine.bin_ceate(z_sim_name, bin_size, -1.0 * lmax, lmax,edge_name)
#_ , zhists_mon = bin_ceate(z_sim_name, bin_size, 0, lmax,edge_name)

# theta of the cylindrical coordinate system
bin_size = np.degrees(np.pi/36) # is 6 degrees
theta_sim_name = cellAttrs.filename.split("ens")[0]+'binFactor1.0ens'+cellAttrs.filename.split("ens")[1]
print(f"Bin size for theta in the cylindrical geometry is set to 1.0*min(cellAttrs.dmon,cellAttrs.dcrowd)={bin_size} in a_m=1 units.")
edge_name = 'thetaEdgesCrd'
thetaedges, thetahists_crd = PipeLine.bin_ceate(theta_sim_name, bin_size, -1*np.degrees(np.pi), np.degrees(np.pi)-bin_size, edge_name)
edge_name = 'thetaEdgesMon'
_ , thetahists_mon = PipeLine.bin_ceate(theta_sim_name, bin_size, -1*np.degrees(np.pi), np.degrees(np.pi)-bin_size, edge_name)

# check if any of the histograms are empty or not.
if any([rhists_crd.any() != 0, rhists_mon.any() != 0, zhists_crd.any() != 0, zhists_mon.any() != 0, thetahists_crd.any() != 0, thetahists_mon.any() != 0]):
    raise ValueError("One of the histogram collectors are not empty!")

for ts in cell.trajectory: # the length of the for loop is equal to number of snapshots (configurations or frames)
    #number density in the cell's frame of reference
    # histogram in r direction
    rpos = np.linalg.norm(crds.positions[:,:2], axis = 1) # r component of position of each crowder
    dummy_hist, _ = np.histogram(rpos, redges)
    rhists_crd = np.add(rhists_crd, dummy_hist)

    rpos = np.linalg.norm(chrm.positions[:,:2], axis = 1) # r component of position of each monomer
    dummy_hist, _ = np.histogram(rpos, redges)
    rhists_mon = np.add(rhists_mon, dummy_hist)

    # histogram in z direction
    zpos = crds.positions[:,2] # z component of position of each crowder
    #zpos = np.abs(crds.positions[:,2]) # absolute value of the z component of position of each crowder - why absolute values? Bacause of PBC along z.
    dummy_hist, _ = np.histogram(zpos, zedges)
    zhists_crd = np.add(zhists_crd, dummy_hist)

    zpos = chrm.positions[:,2] # z component of position of each monomer
    #zpos = np.abs(chrm.positions[:,2]) # absolute value of the z component of position of each monomer - why absolute values? Bacause of PBC along z.
    dummy_hist, _ = np.histogram(zpos, zedges)
    zhists_mon = np.add(zhists_mon, dummy_hist)

    # histogram in theta 
    theta = np.degrees(np.arctan2(crds.positions[:,1], crds.positions[:,0])) # in degrees
    dummy_hist, _ = np.histogram(theta, thetaedges)
    thetahists_crd = np.add(thetahists_crd, dummy_hist)

    theta = np.degrees(np.arctan2(chrm.positions[:,1], chrm.positions[:,0])) # in degrees
    dummy_hist, _ = np.histogram(theta, thetaedges)
    thetahists_mon = np.add(thetahists_mon, dummy_hist)

lastname = 'Crd'
np.savetxt(r_sim_name+'_rHists'+lastname+'.csv', rhists_crd, delimiter = ',')
np.savetxt(z_sim_name+'_zHists'+lastname+'.csv', zhists_crd, delimiter = ',')
np.savetxt(theta_sim_name+'_thetaHists'+lastname+'.csv', thetahists_crd, delimiter = ',')

lastname = 'Mon'
np.savetxt(r_sim_name+'_rHists'+lastname+'.csv', rhists_mon, delimiter = ',')
np.savetxt(z_sim_name+'_zHists'+lastname+'.csv', zhists_mon, delimiter = ',')
np.savetxt(theta_sim_name+'_thetaHists'+lastname+'.csv', thetahists_mon, delimiter = ',')
print('done.')

## distributions around center of geometry of polymer with 0.5 * dynamics fsd as cut_off

In [7]:
%%time
all_data = all_pairs[0][1]
all_trj = all_pairs[0][0]
geometry = 'cylindrical'
rbin_factor = 0.1
zbin_factor = 0.5
print("Setting the name of analyze file...\n")
today = datetime.date.today().strftime('%Y%m%d')
cellAttrs = PipeLine.cellAttributes(all_trj,geometry, splitter='.lammpstrj')
time_unit = 1.0 # time unit = dmon * sqrt(mmon * epmon)
lj_nstep = cellAttrs.bdump # Sampling steps via dump command in Lammps
lj_dt = cellAttrs.dt # Timestep during the sampling process in Lammps
simulation_dt = lj_nstep * lj_dt * time_unit

cell = mda.Universe(all_data, all_trj, format = 'AMIRLAMMPSDUMP', atom_style = "id resid type x y z", dt = simulation_dt)
print(cell.dimensions)
length_step = 10 * max(cellAttrs.dmon, cellAttrs.dcrowd)

#externalRadius = 0.5 * (cellAttrs.dcyl+1)
#print(externalRadius)
#zMax = length_step # the radii of the three types of particle.
#zMin  = -1 * length_step # the radii of the three types of particle.
#near_chrm_cog = cell.select_atoms(f'cyzone {externalRadius} {zMax} {zMin} resid 1', updating=True)
chrm_initial = cell.select_atoms('resid 1')
#near_center = cell.select_atoms(f'prop abs z <= {length_step}', updating=True)

#crds = cell.select_atoms('resid 0') # resid 1 is the atoms creating the chain, resid 0 is crowders
#crds = cell.select_atoms('prop abs z <= 50 resid 1', updating=True) # resid 1 is the atoms creating the chain, resid 0 is crowders
#crds = near_chrm_cog.select_atoms('resid 0', updating=True)
#chrm = cell.select_atoms('resid 1') # resid 1 is the atoms creating the chain, resid 0 is crowders
#chrm = cell.select_atoms('prop abs z <= 50 resid 1', updating=True) # resid 1 is the atoms creating the chain, resid 0 is crowders
#chrm = near_chrm_cog.select_atoms('resid 1', updating=True)
print ("Caution:The histograms are for the cylindrical coordinate system in the geometry=cylindrical. For other goemteries you need to redfined them.")
# Monomer local (number) density: 
# radial direction of the cylindrical coordinate system
#bin_size = cellAttrs.dcrowd / 2.
bin_size = rbin_factor * min(cellAttrs.dmon, cellAttrs.dcrowd)
r_sim_name = cellAttrs.filename.split("ens")[0]+'binFactor'+str(rbin_factor)+'ens'+cellAttrs.filename.split("ens")[1]
print(f"Bin size for r in the cylindrical geometry is set to {rbin_factor}*min(cellAttrs.dmon,cellAttrs.dcrowd)={bin_size} in a_m=1 units.")
lmax = 0.5 * cellAttrs.dcyl
#lmax = 0.5 * (cellAttrs.dcyl - max(cellAttrs.dmon, cellAttrs.dcrowd))
edge_name = 'rEdgesCrd'
redges, rhists_crd = PipeLine.bin_ceate(r_sim_name, bin_size, 0.0, lmax,edge_name)
edge_name = 'rEdgesMon'
_ , rhists_mon = PipeLine.bin_ceate(r_sim_name, bin_size, 0.0, lmax, edge_name)

# z direction of the cylindrical coordinate system
#bin_size = cellAttrs.dcrowd / 2.
bin_size = zbin_factor * min(cellAttrs.dmon, cellAttrs.dcrowd)
z_sim_name = cellAttrs.filename.split("ens")[0]+'binFactor'+str(zbin_factor)+'ens'+cellAttrs.filename.split("ens")[1]
print(f"Bin size for z in the cylindrical geometry is set to {zbin_factor}*min(cellAttrs.dmon,cellAttrs.dcrowd)={bin_size} in a_m=1 units.")
#lmax = length_step
lmax = 0.5 * cellAttrs.lcyl
edge_name = 'zEdgesCrd'
zedges, zhists_crd = PipeLine.bin_ceate(z_sim_name, bin_size, -1.0 * lmax, lmax, edge_name)
#zedges, zhists_crd = bin_ceate(z_sim_name, bin_size, 0, lmax, edge_name)
edge_name = 'zEdgesMon'
_ , zhists_mon = PipeLine.bin_ceate(z_sim_name, bin_size, -1.0 * lmax, lmax,edge_name)
#_ , zhists_mon = bin_ceate(z_sim_name, bin_size, 0, lmax,edge_name)

# theta of the cylindrical coordinate system
bin_size = np.degrees(np.pi/36) # is 6 degrees
theta_sim_name = cellAttrs.filename.split("ens")[0]+'binFactor1.0ens'+cellAttrs.filename.split("ens")[1]
print(f"Bin size for theta in the cylindrical geometry is set to 1.0*min(cellAttrs.dmon,cellAttrs.dcrowd)={bin_size} in a_m=1 units.")
edge_name = 'thetaEdgesCrd'
thetaedges, thetahists_crd = PipeLine.bin_ceate(theta_sim_name, bin_size, -1*np.degrees(np.pi), np.degrees(np.pi)-bin_size, edge_name)
edge_name = 'thetaEdgesMon'
_ , thetahists_mon = PipeLine.bin_ceate(theta_sim_name, bin_size, -1*np.degrees(np.pi), np.degrees(np.pi)-bin_size, edge_name)

# check if any of the histograms are empty or not.
if any([rhists_crd.any() != 0, rhists_mon.any() != 0, zhists_crd.any() != 0, zhists_mon.any() != 0, thetahists_crd.any() != 0, thetahists_mon.any() != 0]):
    raise ValueError("One of the histogram collectors are not empty!")

for ts in cell.trajectory[:100]: # the length of the for loop is equal to number of snapshots (configurations or frames)
    #number density in the cell's frame of reference
    # histogram in r direction
    chrm_fsd =  PipeLine.fsd(chrm_initial.positions)
    #print(max(chrm_initial.positions[:,2])-min(chrm_initial.positions[:,2]))
    zMax = 0.5 * (chrm_fsd+cellAttrs.dmon) # the radii of the three types of particle.
    #zMin  = -0.5 * (chrm_fsd+cellAttrs.dmon) # the radii of the three types of particle.
    #around_chrm = cell.select_atoms(f'cyzone {externalRadius} {zMax} {zMin} resid 1', updating=True)
    around_chrm = cell.select_atoms(f'around {zMax} resid 1', updating=True)
    #near_center = cell.select_atoms(f'prop abs z <= {zMax}', updating=True)
    crds = around_chrm.select_atoms('resid 0', updating=True)
    #crds_2 = around_chrm_2.select_atoms('resid 0', updating=True)
    #crds_1 = near_center.select_atoms('resid 0', updating=True)
    #if crds.positions.shape != crds_2.positions.shape:
     #   print(ts)
      #  print('0 2')
       # print(crds.positions.shape)
        #print(crds_2.positions.shape)
    #if crds.positions.shape != crds_1.positions.shape:
        #print(ts)
       # print('0 1')
       # print(crds.positions.shape)
       # print(crds_1.positions.shape)
    #if crds_1.positions.shape != crds_2.positions.shape:
     #   print(ts)
      #  print('1 2')
       # print(crds_1.positions.shape)
        #print(crds_2.positions.shape)
    
    rpos = np.linalg.norm(crds.positions[:,:2], axis = 1) # r component of position of each crowder
    dummy_hist, _ = np.histogram(rpos, redges)
    rhists_crd = np.add(rhists_crd, dummy_hist)

    rpos = np.linalg.norm(chrm_initial.positions[:,:2], axis = 1) # r component of position of each monomer
    dummy_hist, _ = np.histogram(rpos, redges)
    rhists_mon = np.add(rhists_mon, dummy_hist)

    # histogram in z direction
    zpos = crds.positions[:,2] # z component of position of each crowder
    #zpos = np.abs(crds.positions[:,2]) # absolute value of the z component of position of each crowder - why absolute values? Bacause of PBC along z.
    dummy_hist, _ = np.histogram(zpos, zedges)
    zhists_crd = np.add(zhists_crd, dummy_hist)

    zpos = chrm_initial.positions[:,2] # z component of position of each monomer
    #zpos = np.abs(chrm.positions[:,2]) # absolute value of the z component of position of each monomer - why absolute values? Bacause of PBC along z.
    dummy_hist, _ = np.histogram(zpos, zedges)
    zhists_mon = np.add(zhists_mon, dummy_hist)

    # histogram in theta 
    theta = np.degrees(np.arctan2(crds.positions[:,1], crds.positions[:,0])) # in degrees
    dummy_hist, _ = np.histogram(theta, thetaedges)
    thetahists_crd = np.add(thetahists_crd, dummy_hist)

    theta = np.degrees(np.arctan2(chrm_initial.positions[:,1], chrm_initial.positions[:,0])) # in degrees
    dummy_hist, _ = np.histogram(theta, thetaedges)
    thetahists_mon = np.add(thetahists_mon, dummy_hist)

lastname = 'Crd'
np.savetxt(r_sim_name+'_rHists'+lastname+'.csv', rhists_crd, delimiter = ',')
np.savetxt(z_sim_name+'_zHists'+lastname+'.csv', zhists_crd, delimiter = ',')
np.savetxt(theta_sim_name+'_thetaHists'+lastname+'.csv', thetahists_crd, delimiter = ',')

lastname = 'Mon'
np.savetxt(r_sim_name+'_rHists'+lastname+'.csv', rhists_mon, delimiter = ',')
np.savetxt(z_sim_name+'_zHists'+lastname+'.csv', zhists_mon, delimiter = ',')
np.savetxt(theta_sim_name+'_thetaHists'+lastname+'.csv', thetahists_mon, delimiter = ',')
print('done.')

Setting the name of analyze file...

This is NOT an ALL file, so segnum: 0
[ 31.  31. 759.  90.  90.  90.]
Caution:The histograms are for the cylindrical coordinate system in the geometry=cylindrical. For other goemteries you need to redfined them.
Bin size for r in the cylindrical geometry is set to 0.1*min(cellAttrs.dmon,cellAttrs.dcrowd)=0.1 in a_m=1 units.
Bin size for z in the cylindrical geometry is set to 0.5*min(cellAttrs.dmon,cellAttrs.dcrowd)=0.5 in a_m=1 units.
Bin size for theta in the cylindrical geometry is set to 1.0*min(cellAttrs.dmon,cellAttrs.dcrowd)=5.0 in a_m=1 units.
done.
CPU times: user 11.9 s, sys: 201 ms, total: 12.1 s
Wall time: 12.1 s
